In [ ]:
# Import necessary libraries
import pandas as pd
import pyodbc
from datetime import datetime

# Set up the SQL Server connection
server = 'YOUR_SQL_SERVER_NAME'  # Replace with your SQL Server instance name
database = 'YOUR_DATABASE_NAME'  # Replace with your database name
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')

# Prompt the user for input: Excel file path and sheet name
excel_file = input('Enter the path to your Excel file: ')
sheet_name = input('Enter the sheet name: ')

# Read data from the specified Excel file into a pandas DataFrame
df = pd.read_excel(excel_file, sheet_name=sheet_name, header=2)

# Define the mapping of your Excel columns to the SQL columns
# Replace 'Excel Column Name' with the actual column names in your Excel file
column_mapping = {
    'Excel Column Name 1': 'SQL_Column_Name_1',
    'Excel Column Name 2': 'SQL_Column_Name_2',
    # Add more mappings as needed
}

# Rename columns based on the mapping
df.rename(columns=column_mapping, inplace=True)

# Fill null values in string columns with "null"
string_columns = list(column_mapping.values())  # Use the SQL column names
df[string_columns] = df[string_columns].fillna('null')

# Fill null values in date columns with None (to represent NULL in SQL Server)
date_columns = ['SQL_Column_Name_1', 'SQL_Column_Name_2']  # Use the SQL column names
df[date_columns] = df[date_columns].applymap(lambda x: x if pd.notna(x) else None)

# Define the format of the date in your Excel file
date_format = '%d/%m/%Y'  # Adjust this format to match the format in your Excel file

# Convert date columns to the 'YYYY-MM-DD' format
for column in date_columns:
    df[column] = pd.to_datetime(df[column], format=date_format, errors='coerce')
    df[column] = df[column].dt.strftime('%Y-%m-%d')

# Create a cursor for SQL Server
cursor = conn.cursor()
cursor.fast_executemany = True

# Define the batch size for inserting data into SQL Server
batch_size = 100

# Iterate through the DataFrame and insert data into the SQL Server table
for index, row in df.iterrows():
    try:
        # Construct the SQL INSERT statement dynamically using placeholders
        sql_insert = "INSERT INTO YOUR_TABLE_NAME (" + ", ".join(list(column_mapping.values())) + ") VALUES (" + ", ".join(['?'] * len(column_mapping)) + ")"
        cursor.execute(sql_insert, [row[col] for col in column_mapping.keys()])
        conn.commit()  # Commit the changes to the SQL Server database
    except Exception as e:
        print(f"Error inserting row {index}: {e}")

# Close the cursor and the database connection
cursor.close()

    